In [5]:
import os
import time
from cryptography.fernet import Fernet
# Shared file as the storage
shared_file = 'shared.txt'

# Load the key from the file
def load_key():
    return open('secret.key', 'rb').read()
cipher_suite = Fernet(load_key())

In [6]:
def read_message():
    with open(shared_file, 'rb') as file:
        encrypted_message = file.read()
        # Decrypt the message
        decrypted_message = cipher_suite.decrypt(encrypted_message)
        return decrypted_message.decode()

def receiver():
    last_modified = os.path.getmtime(shared_file)
    while True:
        time.sleep(1)  # Check for changes every second
        current_modified = os.path.getmtime(shared_file)
        if current_modified != last_modified:
            print("Receiver detected a change in the shared file.")
            new_message=read_message()
            print("New message:", new_message)
            if new_message=="Finish!":
                break;
            last_modified = current_modified

In [7]:
receiver()

Receiver detected a change in the shared file.
New message: Covert message
Receiver detected a change in the shared file.
New message: Hello!
Receiver detected a change in the shared file.
New message: Finish!
